In [1]:
print("yashy")

yashy


In [4]:
%pwd

'e:\\medical chatbot\\RAG-based-Medical-Chatbot'

In [3]:
import os
os.chdir("../")

In [10]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from typing import List
from langchain.schema import Document

In [6]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader,
        )

    documents=loader.load()

    return documents

In [7]:
extracted_data = load_pdf_file("data")

In [9]:
print(len(extracted_data))

637


In [11]:
def filter_to_minimal_docs(docs: List[Document]) -> List[Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page_content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append(
            Document(
                page_content=doc.page_content,
                metadata={"source": src}
            )
        )
    return minimal_docs

In [12]:
minimal_docs = filter_to_minimal_docs(extracted_data)

In [ ]:
#Split the Data into Text Chunks
def text_split(minimal_docs):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20, length_function=len)
    text_chunks=text_splitter.split_documents(minimal_docs)
    return text_chunks

In [15]:
text_chunks = text_split(minimal_docs)

print(f"Number of text chunks: {len(text_chunks)}")

Number of text chunks: 5859


In [16]:
#Download the Embeddings from HuggingFace 
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')  #this model return 384 dimensions
    return embeddings

In [17]:
embeddings = download_hugging_face_embeddings()

C:\Users\Yash Khandelwal\AppData\Local\Temp\ipykernel_1860\590304937.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')  #this model return 384 dimensions
e:\anaconda3\envs\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [19]:
vector = embeddings.embed_query("What is the purpose of this trial?")  # Example query to test embeddings
vector

[-0.03778517618775368,
 0.15170325338840485,
 0.00016722497821319848,
 -0.02188190259039402,
 0.08836623281240463,
 0.05721113830804825,
 0.0317489318549633,
 0.012825001031160355,
 0.07328908890485764,
 0.04098068177700043,
 0.0587703101336956,
 0.06926055252552032,
 -0.05780356377363205,
 -0.07808512449264526,
 -0.04693080484867096,
 0.019873902201652527,
 0.06573601812124252,
 0.01087906863540411,
 0.014504792168736458,
 0.019363272935152054,
 -0.049098722636699677,
 -0.04259954020380974,
 0.04426955059170723,
 0.03993979096412659,
 -0.10405206680297852,
 0.03856958448886871,
 -0.05554914474487305,
 -0.015496945939958096,
 0.0686628669500351,
 -0.03543862700462341,
 0.1025443747639656,
 0.036379460245370865,
 0.022359415888786316,
 0.022364994511008263,
 0.014204142615199089,
 -0.043799158185720444,
 0.003482842817902565,
 -0.007823486812412739,
 -0.03559206426143646,
 0.013100344687700272,
 0.03448628634214401,
 -0.07137159258127213,
 -0.05834125354886055,
 0.054483551532030106,
 -

In [20]:
print(len(vector))

384


In [21]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [23]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["OPENROUTER_API_KEY"] = OPENROUTER_API_KEY

In [24]:
from pinecone import Pinecone
pinecone_api_key = PINECONE_API_KEY

pc = Pinecone(api_key=pinecone_api_key)


In [25]:
pc

In [29]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=pinecone_api_key)

index_name = "medical-chatbot"

# Check if index exists
if index_name not in [idx['name'] for idx in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=384,  # dimension of your embeddings
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pc.Index(index_name)


In [30]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
    
)

In [31]:
# load existing index from the pinecone server

from langchain_pinecone import PineconeVectorStore
#embed each chunk and upsert the embedding into your pinecone index

decsearch = PineconeVectorStore.from_existing_index(
    index_name = index_name,
    embedding = embeddings
)

In [33]:
retriever = docsearch.as_retriever(search_type = "similarity", search_kwargs = {"k":3})

In [35]:
retrieved_docs = retriever.invoke("what is Acne?")
retrieved_docs

[Document(id='0d9b3911-7db4-44ce-9d6f-7901c5017b75', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='c342a5d9-3592-4760-841b-397d588e2ac4', metadata={'source': 'data\\Medical_book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed. (Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='6353fe2c-4852-4473-a4e8-ac26563ab27d', metadata={'source': 'data\\Medical_book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the face, chest, and back. It occurs when the\npores of the skin become clogged wi

In [37]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
import os
from langchain_openai import ChatOpenAI

load_dotenv()


OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")
if not OPENROUTER_API_KEY:
    raise ValueError("OpenRouter API key not found. Please set it in the .env file.")

llm = ChatOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    model="mistralai/mistral-7b-instruct",  # You can also try 'mistral-large'
    temperature=0.5
)

In [38]:
system_prompt = (
    "you are an medical assistant for question answering tasks. "
    "use the following pieces of retrieved context to answer "
    "the question . if you dont know the answer , say that you dnt know "
    "use three sentences maximum and keep the answer concise"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
    ("system", system_prompt),
    ("human", "{input}")
    ]
)

In [39]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [42]:
response = rag_chain.invoke({"input": "what is acne"})
print(response["answer"])

 Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. Acne vulgaris, the medical term for common acne, is the most common skin disease.
